In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from nilearn.connectome import sym_matrix_to_vec
from scipy.stats import pearsonr
from cmath import isinf
import torch.nn.functional as F
#import seaborn as sns
from torch.utils.data import Dataset, DataLoader, Subset, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import math
from cmath import isinf
from utils_v import compute_target_score
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, KFold, LearningCurveDisplay, learning_curve
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from helper_classes import MatData, MLP
#from dev_losses import cauchy, rbf, gaussian_kernel, CustomSupCon, CustomContrastiveLoss
#from losses import KernelizedSupCon

In [3]:
def multivariate_kernel(d, sigma : float,):
    
    exponent = -0.5 *(d/sigma)  
    exp = torch.exp(exponent)
    
    return exp

In [4]:
def threshold(features, threshold, regions=None):
    threshold = np.quantile(features, threshold)
    features_thresholded = np.where(np.abs(features) > threshold, 0, features)
    norm = np.linalg.norm(features_thresholded)
    normalized_features = features_thresholded / norm if norm != 0 else features_thresholded
    return normalized_features

In [5]:
path_matrix = "/data/parietal/store/work/dwassermann/data/victoria_mat_age/matrices.npy"
matrix = np.load(path_matrix)
vec_m = sym_matrix_to_vec(matrix, discard_diagonal=True)
vec_m_thresholded = threshold(vec_m ,0.5)

In [10]:
def evaluate_augmentation_wrt_original(matrix, augmented_matrix, sigma=100000):
    
    matrix = torch.tensor(matrix)
    augmented_matrix = torch.tensor(augmented_matrix)
    
    #compute the average value of the kernel of original samples and their augmented counterpart
    augmented_dist = torch.cdist(augmented_matrix, matrix,p=2)**2
    augmented_kernel = multivariate_kernel(augmented_dist, sigma)
    avg_kernel_values_diag = torch.mean(torch.diag(augmented_kernel))
    
    #compute average value of original samples with respect to all other original samples 
    original_dist = torch.cdist(matrix, matrix, p=2)**2
    original_kernel = multivariate_kernel(original_dist, sigma)
    original_kernel = original_kernel.numpy()
    avg_kernel_non_diag_values = np.mean(sym_matrix_to_vec(original_kernel, discard_diagonal=True))
    
    return avg_kernel_values_diag > avg_kernel_non_diag_values

In [9]:
evaluate_augmentation_wrt_original(vec_m, vec_m_thresholded)

tensor(True)

In [11]:
def evaluate_augmentation_wrt_augmented(matrix, augmented_matrix, sigma=100000):
    
    matrix = torch.tensor(matrix)
    augmented_matrix = torch.tensor(augmented_matrix)
    
    #compute the average value of the kernel of original samples and their augmented counterpart
    augmented_dist = torch.cdist(augmented_matrix, matrix,p=2)**2
    augmented_kernel = multivariate_kernel(augmented_dist, sigma)
    avg_kernel_values_diag = torch.mean(torch.diag(augmented_kernel))
    
    #compute average kernel value of augmentations with respect to the other augmentations 
    augmented_kernel_eval = augmented_kernel.numpy()
    avg_kernel_non_diag_values = np.mean(sym_matrix_to_vec(augmented_kernel_eval, discard_diagonal=True))
    
    return avg_kernel_values_diag > avg_kernel_non_diag_values

In [13]:
evaluate_augmentation_wrt_augmented(vec_m, vec_m_thresholded)

tensor(True)